
#              Business Geolocation in Bogota Colombia

## Table of Contents





### 1. Introduction: Business Problem

### 2. Data

### 3. Methodology

### 4. Results and Discussion 

### 5. Conclusions 




<a id='item1'></a>

## 1. Introduction: Business Problem



Bogotá is the capital of the Republic of Colombia. In 2018 it had an estimated population of 7.4 million people.
It is one of the most important cities in Latin America both in number of inhabitants and in GDP. The city has a dynamic economy,according to data from the Chamber of Commerce of Bogotá 35855 new businesses were created  in 2018, although it is estimated that only 30% survived a year after its creation.

Given this scenario, it is very important for people interested in creating new businesses to have as much information as possible about structures and the behavior of the markets in which they want to enter. 

### Business Problem
This analysis aims to expand knowledge through statistical tools about the geolocation of the different types of businesses throughout the 20 locations in which the city of Bogota is divided. In this way and depending on the particular analysis of each investor you can decide in which location you can establish your business, this will depend on many factors because some people prefer to take advantage of the agglomeration of companies, while others prefer to establish their businesses in places where There is not much competition among other important factors to consider.



<a id='item2'></a>

<a id='item2'></a>

## 2. Data



In order to meet the above objectives, we will need the following data:

- List of "Localidades" of the Bogota, city.

- Latitude and Longitude coordinates of those territorial subdivisions. This information is required to plot the map.

- List of all VenueCategories in the "Localidades" in Bogota.This information is required in order to clustering the             localidades.

### Sources of data.

- The list of "Localidades" is in the page https://es.wikipedia.org/wiki/Bogot%C3%A1, the table it will extracted with           Pandas'method read_html.

- The Latitude and Longitude  coordinates of localidades will get with Python Geocoder Package.

- The List of VenueCategories it will get of Foursquare Api.
 
 


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Extracting Localiades from Wikipedia

In [2]:

url = "https://es.wikipedia.org/wiki/Anexo:Localidades_de_Bogot%C3%A1"
locs = pd.read_html(url)

loc_bog = locs[0]
loc_bog = loc_bog.drop(["Nº"], axis=1)
loc_bog.head()

,Localidad,Códigos Postales,Superficie km²[2]​,Población[3]​,Densidad hab/km²
0,Usaquén,110111-110151,65.31,501 999,7 686.4
1,Chapinero,110211-110231,38.15,139 701,3 661.88
2,Santa Fe,110311-110321,45.17,110 048,2 436.3
3,San Cristóbal,110411-110441,49.09,404 697,8 243.98
4,Usme,110511-110571,215.06,457 302,2 126.39


In [3]:
#localidades
localidades = loc_bog["Localidad"].to_frame()
localidades.head()

,Localidad
0,Usaquén
1,Chapinero
2,Santa Fe
3,San Cristóbal
4,Usme


### Getting Coordinates

In [4]:
# define a function to get coordinates
def get_latlng(localidad):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bogota, Colombia'.format(localidad))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
### call the function to get the coordinates, store in a new list using list comprehension
import warnings
warnings.filterwarnings("ignore")

coords = [ get_latlng(localidad) for localidad in localidades["Localidad"].tolist() ]


In [6]:
coords

[[4.692590000000052, -74.03008999999997],
 [4.638480000000072, -74.06020999999998],
 [4.594590000000039, -74.06404999999995],
 [4.576430000000073, -74.09313999999995],
 [4.4982800000000225, -74.10744999999997],
 [4.561820000000068, -74.12733999999995],
 [4.609740000000045, -74.18279999999999],
 [4.627480000000048, -74.17021999999997],
 [4.686370000000068, -74.15099999999995],
 [4.701270000000022, -74.11268999999999],
 [4.734380000000044, -74.08562999999998],
 [4.669710000000066, -74.07784999999996],
 [4.623290000000054, -74.07224999999994],
 [4.604310000000055, -74.08978999999994],
 [4.596580000000074, -74.11201999999997],
 [4.633340000000032, -74.10627999999997],
 [4.594370000000026, -74.07688999999993],
 [4.624070000000074, -74.06613999999996],
 [4.553670000000068, -74.14647999999994],
 [4.554740000000038, -74.14691999999997]]

In [7]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
coords_bog = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
coords_bog.head(3)

,Latitude,Longitude
0,4.69259,-74.03009
1,4.63848,-74.06021
2,4.59459,-74.06405


In [9]:
# merge the coordinates into the original dataframe
localidades['Latitude'] = coords_bog['Latitude']
localidades['Longitude'] = coords_bog['Longitude']


In [10]:
localidades.head()

,Localidad,Latitude,Longitude
0,Usaquén,4.69259,-74.03009
1,Chapinero,4.63848,-74.06021
2,Santa Fe,4.59459,-74.06405
3,San Cristóbal,4.57643,-74.09314
4,Usme,4.49828,-74.10745


### Bogota's Map

In [12]:
address = 'Bogota,Colombia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bogota,Colombia {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Bogota,Colombia 4.59808, -74.0760439.


In [14]:
# create map of Toronto using latitude and longitude values
map_bog = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, localidad in zip(localidades['Latitude'], localidades['Longitude'], localidades['Localidad']):
    label = '{}'.format(localidad)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bog)  
    
map_bog

In [42]:
# save the map as jpg file
map_bog.save('map_bog.png')

<a id='item3'></a>

## Methodology


To achieve the objective of this project, we will now list the VenuesCategories generated by Foursquare API within a limit of 100 meters around the specified coordinates with a limit of 100. We will generate a DataFrame in which the name of each Venue is specified, its category, Your Latitude and Longitude, then we will count how many you get were obtained by each of the locations, in addition to finding out how many different VenuesCategories returned the query to the API.

The above information will be useful to construct a DataFrame in which it is possible to identify if a type of Venue Categorie exists in a locality or not. Then a classification of the 5 VenuesCategories or types of businesses more frequent in each of the localities will be made and finally, the K-Means algorithm will be used to generate 5 clusters in which the territorial units of Bogota are segmented according to the most common type of business in each of them. To obtain a better understanding of the results obtained, we will generate a map through the Folium package, in which the different clusters are clearly identified.

In [15]:
# define Foursquare Credentials and Version
CLIENT_ID = '5UBHRHUNLSGAW5BJCGGTT1DAH3TIUQGWRTNGYJOPCCCNPNOT' # your Foursquare ID
CLIENT_SECRET = 'FR21KJSDPXXNKU2MLSWA2XUBABPPTZTMHYCVXHPLENCBJZM3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5UBHRHUNLSGAW5BJCGGTT1DAH3TIUQGWRTNGYJOPCCCNPNOT
CLIENT_SECRET:FR21KJSDPXXNKU2MLSWA2XUBABPPTZTMHYCVXHPLENCBJZM3


In [17]:
radius = 3000
LIMIT = 100

venues = []

for lat, long, localidad in zip(localidades['Latitude'], localidades['Longitude'], localidades['Localidad']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            localidad,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [18]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Localidad', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1420, 7)


,Localidad,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Usaquén,4.69259,-74.03009,La Puerta De Alcalá,4.694399,-74.029996,Spanish Restaurant
1,Usaquén,4.69259,-74.03009,Parque Usaquén,4.695163,-74.030927,Park
2,Usaquén,4.69259,-74.03009,WeWork Usaquén,4.694304,-74.032745,Coworking Space
3,Usaquén,4.69259,-74.03009,WOK Usaquén,4.694421,-74.031616,Asian Restaurant
4,Usaquén,4.69259,-74.03009,La Provence,4.694381,-74.030867,French Restaurant


### Let's check how many venues were returned for each neighborhood

In [19]:
venues_df["Localidad"].value_counts().to_frame()

,Localidad
Teusaquillo,100
Los Mártires,100
Chapinero,100
Rafael Uribe Uribe,100
Puente Aranda,100
Santa Fe,100
Engativá,100
Suba,100
Antonio Nariño,100
Usaquén,100


### Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 178 uniques categories.


In [21]:
# print out the list of categories
venues_df['VenueCategory'].value_counts().to_frame()

,VenueCategory
Restaurant,93
Café,75
Park,54
Coffee Shop,53
Hotel,44
Burger Joint,42
Pizza Place,41
Bakery,40
Italian Restaurant,30
BBQ Joint,29


### Analyze Each Neighborhood

In [22]:

# one hot encoding
bog_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bog_onehot['Localidad'] = venues_df['Localidad'] 

# move neighborhood column to the first column
fixed_columns = [bog_onehot.columns[-1]] + list(bog_onehot.columns[:-1])
bog_onehot = bog_onehot[fixed_columns]

print(bog_onehot.shape)
bog_onehot.head()

(1420, 179)


,Localidad,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Drugstore,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Theme Restaurant,Trade School,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Usaquén,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Usaquén,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Usaquén,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Usaquén,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Usaquén,0,0,0,0,0,0,0,0,0,

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
bog_grouped = bog_onehot.groupby(["Localidad"]).mean().reset_index()


bog_grouped

,Localidad,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Drugstore,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Theme Restaurant,Trade School,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Wine Bar,Wings Joint,Women's Store
0,Antonio Nariño,0.000000,0.000,0.000000,0.00,0.00,0.01,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.030000,0.050000,0.030000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.01,0.00,0.000000,0.000000,0.00,0.00,0.050000,0.00,0.0,0.010000,0.010000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.010000,0.00,0.00,0.00,0.000000,0.010000,0.00,0.00,0.010000,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.02,0.00,0.010000,0.010000,0.01,0.00,0.00,0.00,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.020000,0.010000,0.010000,0.030000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.020000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.00,0.02,0.00,0.000000,0.00,0.000000,0.030000,0.00,0.000000,0.00,0.00,0.030000,0.00,0.010000,0.000000,0.000000,0.0,0.000000,0.010000,0.000000,0.00,0.010000,0.00,0.00,0.000000,0.060000,0.00,0.00,0.00,0.00,0.02,0.010000,0.090000,0.00,0.010000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.060000,0.00,0.00,0.00,0.010000,0.050000,0.00,0.050000,0.020000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.010000,0.00,0.01,0.01,0.00,0.020000,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.01,0.00
1,Barrios Unidos,0.000000,0.000,0.000000,0.00,0.01,0.00,0.00,0.01,0.010000,0.00,0.00,0.020000,0.00,0.030000,0.040000,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.030000,0.00,0.00,0.000000,0.010000,0.00,0.00,0.010000,0.00,0.0,0.030000,0.000000,0.000000,0.00,0.00,0.01,0.000000,0.01,0

In [24]:
print(bog_grouped.shape)

(20, 179)


### Top 5 Venues Categories

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Localidad']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
locals_venues_sorted = pd.DataFrame(columns=columns)
locals_venues_sorted['Localidad'] = bog_grouped['Localidad']

for ind in np.arange(bog_grouped.shape[0]):
    locals_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bog_grouped.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)

locals_venues_sorted    

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Antonio Nariño,Pizza Place,Restaurant,Park,Burger Joint,Sandwich Place
1,Barrios Unidos,Coffee Shop,Bakery,Park,Restaurant,Bowling Alley
2,Bosa,Grocery Store,Park,Department Store,Historic Site,Restaurant
3,Chapinero,Coffee Shop,Restaurant,Bakery,Café,Hotel
4,Ciudad Bolívar,Park,Gym,Burger Joint,Breakfast Spot,Electronics Store
5,Engativá,Café,Burger Joint,Sandwich Place,Hotel,Fast Food Restaurant
6,Fontibón,Café,Airport Lounge,Coffee Shop,Airport,Hotel
7,Kennedy,Gym,Park,Restaurant,Burger Joint,Pizza Place
8,La Candelaria,Café,Restaurant,History Museum,Hotel,Nightclub
9,Los Mártires,Restaurant,Café,History Museum,Nightclub,Hotel


### CLUSTERING

In [27]:
# set number of clusters
kclusters = 5

bogota_grouped_clustering = bog_grouped.drop('Localidad', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bogota_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 0, 1, 0, 0, 3, 0, 0])

In [30]:
# add clustering labels
locals_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bog_merged = localidades.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bog_merged = bog_merged.join(locals_venues_sorted.set_index('Localidad'), on='Localidad')

bog_merged.head() # check the last columns!

,Localidad,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Usaquén,4.69259,-74.03009,0,Hotel,Coffee Shop,Café,Park,French Restaurant
1,Chapinero,4.63848,-74.06021,0,Coffee Shop,Restaurant,Bakery,Café,Hotel
2,Santa Fe,4.59459,-74.06405,0,Restaurant,Café,History Museum,Hotel,Nightclub
3,San Cristóbal,4.57643,-74.09314,3,Restaurant,Pizza Place,Park,BBQ Joint,Burger Joint
4,Usme,4.49828,-74.10745,2,Bus Station,Cocktail Bar,Pool,Mountain,Dog Run


In [31]:
bog_merged.shape

(20, 9)

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bog_merged['Latitude'],bog_merged['Longitude'],bog_merged['Localidad'], bog_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:

# save the map as jpg file
map_clusters.save('map_clusters.png')

### CLUSTER 1

In [33]:
bog_merged.loc[bog_merged['Cluster Labels'] == 0, bog_merged.columns[[0] + list(range(4, bog_merged.shape[1]))]]

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Usaquén,Hotel,Coffee Shop,Café,Park,French Restaurant
1,Chapinero,Coffee Shop,Restaurant,Bakery,Café,Hotel
2,Santa Fe,Restaurant,Café,History Museum,Hotel,Nightclub
8,Fontibón,Café,Airport Lounge,Coffee Shop,Airport,Hotel
9,Engativá,Café,Burger Joint,Sandwich Place,Hotel,Fast Food Restaurant
11,Barrios Unidos,Coffee Shop,Bakery,Park,Restaurant,Bowling Alley
12,Teusaquillo,Restaurant,Café,Coffee Shop,Park,Italian Restaurant
13,Los Mártires,Restaurant,Café,History Museum,Nightclub,Hotel
15,Puente Aranda,Restaurant,Hotel,Bakery,Latin American Restaurant,Seafood Restaurant
16,La Candelaria,Café,Restaurant,History Museum,Hotel,Nightclub


### CLUSTER 2

In [35]:
bog_merged.loc[bog_merged['Cluster Labels'] == 1, bog_merged.columns[[0] + list(range(4, bog_merged.shape[1]))]]

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,Ciudad Bolívar,Park,Gym,Burger Joint,Breakfast Spot,Electronics Store
19,Sumapaz,Park,Gym,Burger Joint,Breakfast Spot,Electronics Store


### CLUSTER 3

In [36]:
bog_merged.loc[bog_merged['Cluster Labels'] == 2, bog_merged.columns[[0] + list(range(4, bog_merged.shape[1]))]]

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Usme,Bus Station,Cocktail Bar,Pool,Mountain,Dog Run


### CLUSTER 4

In [38]:
bog_merged.loc[bog_merged['Cluster Labels'] == 3, bog_merged.columns[[0] + list(range(4, bog_merged.shape[1]))]]

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,San Cristóbal,Restaurant,Pizza Place,Park,BBQ Joint,Burger Joint
6,Bosa,Grocery Store,Park,Department Store,Historic Site,Restaurant
7,Kennedy,Gym,Park,Restaurant,Burger Joint,Pizza Place
10,Suba,Burger Joint,Movie Theater,Pizza Place,Wings Joint,Fast Food Restaurant
14,Antonio Nariño,Pizza Place,Restaurant,Park,Burger Joint,Sandwich Place


### CLUSTER 5

In [39]:
bog_merged.loc[bog_merged['Cluster Labels'] == 4, bog_merged.columns[[0] + list(range(4, bog_merged.shape[1]))]]

,Localidad,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Tunjuelito,Park,Construction & Landscaping,Department Store,Fast Food Restaurant,Cajun / Creole Restaurant


<a id='item4'></a>

### 4. Results and Discussion

Cluster number 1 is made up of Usaquen, Chapinero, Santa Fe, Fontibon, Barrios Unidos, Teusaquillo, Los Martires, Puente Aranda, La Candelaria and Rafael Uribe Uribe. The VenuesCategories that characterize this group are Restaurants, Hotels, Cafeterias, Coffe Shops, Bakeries and Hamburgers.


Cluster number 2 is formed by the towns of Sumapaz and Ciudad Bolivar.
The types of places Categories that characterize this group are Parks,Gyms, Burguer Joints and Breakfast Spot.

Cluster 3 is formed only by the locality of Usme. In this cluster a bus station was obtained as the most representative VenueCategory, then by CoctailBar, Pool, Mountain and DogRun

Cluster 4 groups the towns of San Cristóbal, Bosa, Kennedy, Suba and Antonio Nariño. The main venues Categories in restaurants, gyms, supermarkets, pizzerias, cinemas and Grocery stores.

Cluster 5 is formed only by the town of Tunjuelito and the Top 5 VenuesCategories are Park, Construction, Department Store, Fast Food Restaurant and Cajun Creole Restaurant.

Cluster 2 and 3 do not have much variety of VenuesCategories, it should be noted that the town of Sumapaz located in Cluster 2 is above all a rural area with the lowest population density rate in the city.

<a id='item5'></a>

### 5. Conclusions


- When deciding in which location to establish a business and depending on whether you want to take advantage of agglomeration economies or if you want to avoid a lot of competition. We have that Restaurants, Hotels and Cafes, are located mainly in cluster 1 and especially in the towns of Usaquen, Chapinero, Teusaquillo and Los Martires.

- Gyms, supermarkets, pizzerias and vegetable stores stand out in cluster 4, mainly in the towns of Bosa, San Cristobal and Suba.